# Horovod を使った分散学習
Horovod は MPI を使った分散学習のためのフレームワークです。より詳細な情報は [Horovod README](https://github.com/uber/horovod) をご確認下さい。

Amazon SageMaker では Horovod の活用もスクリプトを少し変更するだけで可能です。

## Horovod による分散学習を行う学習スクリプトの作成

#### **オリジナルの学習スクリプトである`training_script/cifar10_keras_sm.py`をコピーした上で、`training_script/cifar10_keras_dist.py.`として保存して下さい。新しいファイルを書き換え用のファイルとして使います。**


の中において、

#### ① Horovod の設定

Horovod を使用するように `main()` 関数の中に 下記を追加します。



```python
    import horovod.keras as hvd
    hvd.init()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.visible_device_list = str(hvd.local_rank())
    K.set_session(tf.Session(config=config))
```

#### ② Callbacks の設定

Horovod へ対応するために `main()` 関数の中に callbacks を追加します。

```python
    callbacks.append(hvd.callbacks.BroadcastGlobalVariablesCallback(0))
    callbacks.append(hvd.callbacks.MetricAverageCallback())
    callbacks.append(hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1))
```

また、checkpoint と tensorboard の callback がシングルプロセスのログだけ送信するように変更します。 

```python
    if hvd.rank() == 0:
        callbacks.append(ModelCheckpoint(args.output_dir + '/checkpoint-{epoch}.h5'))
        callbacks.append(TensorBoard(log_dir=args.model_output_dir,update_freq='epoch'))
```

#### ③ Horovod に対応した Optimizer の設定変更
Horovod へ対応するために keras_model_fn へ `hvd` 引数を追加します。

```python
#  hvd を追加します。
def keras_model_fn(learning_rate, weight_decay, optimizer, momentum, hvd): 
```
また、model インスタンスを作成する際に、`hvd` を引数に渡します。
```python
model = keras_model_fn(args.learning_rate, args.weight_decay, args.optimizer, args.momentum, hvd)
```

その、`keras_model_fn()` 関数の中において`size=1` を `size=hvd.size()`　へ変更。 

さらに、
```python
 model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
```
の直前に

```python
opt = hvd.DistributedOptimizer(opt)
```
を追加。

## 分散学習の実行
To start a distributed training job with Horovod, configure the job distribution:
```python
distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': # Number of Horovod processes per host
                        }
                }
```

Run the same job using 2 ml.p3.2xlarge instances (processes_per_host:1).  
add the distributions configuration

In [5]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()


dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-us-east-1-815969174475/data/DEMO-cifar10'

In [14]:
from sagemaker.tensorflow import TensorFlow

distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': 1}
                }

# Change base_job_name to 'cifar10-dist' for console visibility
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_sample3.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 10},
                       train_instance_count=2,
                       train_instance_type='ml.p3.2xlarge',
                       distributions=distributions
                      )

In [15]:
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2019-10-01 02:54:01 Starting - Starting the training job...
2019-10-01 02:54:03 Starting - Launching requested ML instances......
2019-10-01 02:55:07 Starting - Preparing the instances for training......
2019-10-01 02:56:19 Downloading - Downloading input data...
2019-10-01 02:56:56 Training - Training image download completed. Training in progress..2019-10-01 02:56:54,159 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-10-01 02:56:57,471 sagemaker-containers INFO     Starting MPI run as worker node.
2019-10-01 02:56:57,472 sagemaker-containers INFO     Waiting for MPI Master to create SSH daemon.
2019-10-01 02:56:57,473 sagemaker-containers INFO     Cannot connect to host algo-1
2019-10-01 02:56:57,473 sagemaker-containers INFO     Connection failed with exception: 
 [Errno None] Unable to connect to port 22 on 10.2.224.32
2019-10-01 02:56:58,474 sagemaker-containers INFO     Cannot connect to host algo-1
2019-10-01 02:56:58,474 sagemaker-

**Good job!**  
You can now use SageMaker training jobs for distributed jobs.
Before continuing to the next notebook, look at the distribution job metrics from CloudWatch and TensorBoard.  
You can use TensorBoard to compare between the different jobs that you ran.
Run TensorBoard with   
`--logdir dist:dist_model_dir,pipe:pipe_model_dir,file:normal_job_model_dir`